In [3]:
import os
import gc
import csv
import argparse
import pickle

import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

from dataset import ICUSepsisDataset
from model import SepsisPredictionModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

OPTIMIZERS = {
    'adam': torch.optim.Adam,
    'adadelta': torch.optim.Adadelta,
    'sgd': torch.optim.SGD,
    'adagrad': torch.optim.Adagrad
}

argparser = argparse.ArgumentParser(description='Train mask detection nerual network')
argparser.add_argument('--data-path', type=str, required=True, dest='data_path', default='/home/student/data/')
argparser.add_argument('--epochs', type=int, dest='epochs', default=100)
argparser.add_argument('--optimizer', type=str, dest='optimizer', choices=OPTIMIZERS.keys(), default='adam')
argparser.add_argument('--lr', type=float, dest='lr', default=0.005)
argparser.add_argument('--print-steps', type=int, dest='print_steps')

args = argparser.parse_args(['--data-path', '/Users/snirlugassy/Projects/sepsis_early_detection/data', '--epochs', '1'])

print('====== TRAIN =======')
print('optimizer:', args.optimizer)
print('epochs:', args.epochs)
print('l-rate:', args.lr)
print('device:', device)
print('====================')

model = SepsisPredictionModel(input_size=len(ICUSepsisDataset.features))
model.to(device)

icu_train = ICUSepsisDataset(os.path.join(args.data_path, 'train'))
train_loader = DataLoader(icu_train, batch_size=1, shuffle=True)

loss = torch.nn.CrossEntropyLoss()
optimizer = OPTIMIZERS[args.optimizer](model.parameters(), lr=args.lr)
 

====== TRAIN =======
optimizer: adam
epochs: 1
l-rate: 0.005
device: cpu
Found 20000 files in /Users/snirlugassy/Projects/sepsis_early_detection/data/train


In [5]:
x,y = next(iter(train_loader))
print(x.shape, y.shape)
output = model(x)
print(output.shape)

torch.Size([1, 45, 9]) torch.Size([1, 45])
torch.Size([1, 45, 2])


In [10]:
loss()

tensor(0.6885, grad_fn=<NllLossBackward0>)